**NOTEBOOK USED TO GENERATE SENTENCE EMBEDDINGS WHICH ARE THEN USED FOR COSINE SIMILARITY AND TRAINING OTHER REGRESSORS WITH AND WITHOUT PREVIOUS FEATURES DESCRIBED IN THE REPORT.**

In [ ]:
import numpy as np

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 6.0MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
     |████████████████████████████████| 2.9MB 63.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=932f572b112ac13fb97db19418271584578e4cdd19577be7f55f8d86f6fc14de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
project_folder = "./drive/My Drive/csc2515-project/"

In [ ]:
import pandas as pd
import os
import numpy as np

# Load dataset
#questions_answers = pd.read_csv(os.path.join(project_folder, "Train.csv"))
questions_answers = pd.read_csv(os.path.join(project_folder, "Train_RemovedNAN_AddedNormScore.csv"))

print(questions_answers.head(5).to_string())

   Unnamed: 0  Score  QId  Before  After  Cosine_to_Question  Cosine_to_Answers  Word_cnt  Char_cnt  Avg_char_length  Urls  Codes  Grade_Level  Dale_chall  Reading_Ease  Polarity  Subjectivity  Cumulative_ Answer_Score  Num_Answers  Num_Questions  Average_Answer_Score  Q_Word_Cnt  Q_Char_Cnt  Q_Avg_char_length  Q_Urls  Q_CodeSections  Q_GradeLevel  Q_Dale_chall  Q_ReadingEase  Q_Polarity  Q_Subjectivity                                                                                                                                                                                                             Clean_Question                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
# Drop the Nan's
questions_answers['Clean_Question'].replace('', np.nan, inplace=True)
questions_answers.dropna(subset=['Clean_Question'], inplace=True)

questions_answers['Clean_Answer'].replace('', np.nan, inplace=True)
questions_answers.dropna(subset=['Clean_Answer'], inplace=True)

print(questions_answers.shape)

# Drop duplicate best answers
questions_answers = questions_answers.groupby('QId').filter(lambda x: x.nlargest(2, 'Score')['Score'].iloc[1]!=x.nlargest(2, 'Score')['Score'].iloc[0])
questions_answers.reset_index(drop=True, inplace=True)

# Normalize scores
max = questions_answers.groupby('QId')['Score'].transform('max')
questions_answers['Normalized_Score'] = questions_answers['Score'].div(max)

(108705, 34)


In [ ]:
# Sentence embedding without fine tuning
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.3MB 7.3MB/s 
     |████████████████████████████████| 2.9MB 41.6MB/s 
     |████████████████████████████████| 1.1MB 47.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101034 sha256=2631302e38344dd660e442e612e3169ab6a2306c3f1124108751a9bf9c742b3b
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
Successfully built sentence-transformers
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Found existing installation: transformers 4.0.1
    Uninstalling transformers-4.0.1:
      Successfully uninstalled transformers-4.0.1


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

100%|██████████| 1.24G/1.24G [00:52<00:00, 23.7MB/s]


**SENTENCE EMBEDDING WITHOUT IMPOSING SEQUENCE LENGTH**

In [ ]:
# Choose tensor or NP style embeddings as output
#q_sentence_embeddings = model.encode(questions_answers['Clean_Question'], batch_size=32)
q_sentence_embeddings = model.encode(questions_answers['Clean_Question'], batch_size=32, convert_to_tensor=True)

In [ ]:
# Choose tensor or NP style embeddings as output
a_sentence_embeddings = model.encode(questions_answers['Clean_Answer'], batch_size=32)
a_sentence_embeddings = model.encode(questions_answers['Clean_Answer'], batch_size=32, convert_to_tensor=True)

In [ ]:
# Store sentences & embeddings on disc
import pickle
import os

with open(os.path.join(project_folder, 'q_embeddings_tensor.pkl'), "wb") as fOut:
    pickle.dump({'sentences': questions_answers['Clean_Question'], 'embeddings': q_sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(project_folder, 'a_embeddings_tensor.pkl'), "wb") as fOut:
    pickle.dump({'sentences': questions_answers['Clean_Answer'], 'embeddings': a_sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# CHOOSE WHETHER TO LOAD TENSOR OR NP ARRAY
# Load the sentence embeddings from disk
import pickle
import os

with open(os.path.join(project_folder, 'q_embeddings_tensor.pkl'), "rb") as fIn:
    stored_data = pickle.load(fIn)
    q_sentences = stored_data['sentences']
    q_sentence_embeddings = stored_data['embeddings']

# Load the sentence embeddings from disk
with open(os.path.join(project_folder, 'a_embeddings_tensor.pkl'), "rb") as fIn:
    stored_data = pickle.load(fIn)
    a_sentences = stored_data['sentences']
    a_sentence_embeddings = stored_data['embeddings']

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import pandas as pd

questions_answers = pd.read_csv(os.path.join(project_folder, 'Train_RemovedNAN_AddedNormScore.csv'))
# Find the split index, then iterate to next question since the the split should not be in the middle of a question
split_index = int(len(questions_answers) * 0.8)
split_Q = questions_answers['QId'].iloc[split_index]
while(split_index<len(questions_answers) and questions_answers['QId'].iloc[split_index] == split_Q):
  split_index += 1
questions_answers_train = questions_answers[:split_index]
questions_answers_test = questions_answers[split_index:]

In [ ]:
questions_answers_test = questions_answers_test.reset_index()

In [ ]:
q_sentence_embeddings_val = q_sentence_embeddings[split_index:]
a_sentence_embeddings_val = a_sentence_embeddings[split_index:]

In [ ]:
q_sentence_embeddings_val.shape

torch.Size([21740, 1024])

In [ ]:
print(pytorch_cos_sim(q_sentence_embeddings[0:1], a_sentence_embeddings[0:1]))

tensor([[0.4619]])


**COSINE SIMILARITY AND ACCURACY MEASURE**

In [ ]:
from sentence_transformers.util import pytorch_cos_sim

groups_df = pd.DataFrame(columns=questions_answers.columns)
groups_df['Cosine_Sim'] = None

groups = questions_answers.groupby('QId')
cosine_scores = []
for name, group in groups:

  for idx, row in group.iterrows():
    #print(row)
    cosine_sim = pytorch_cos_sim(q_sentence_embeddings[idx], a_sentence_embeddings[idx])
    cosine_sim = cosine_sim.numpy()
    cosine_sim = cosine_sim.item()
    cosine_scores.append(cosine_sim)

print(len(cosine_scores))
print(cosine_scores)

questions_answers['Cosine_Sim'] = cosine_scores

108705
[0.4618881344795227, 0.49041903018951416, 0.27797064185142517, 0.36816683411598206, 0.29395791888237, 0.3313694894313812, 0.3369269073009491, 0.3202351927757263, 0.2749350666999817, 0.5894733667373657, 0.7393924593925476, 0.6440446972846985, 0.7635807991027832, 0.6465815305709839, 0.7295071482658386, 0.5364860892295837, 0.6607780456542969, 0.6880965232849121, 0.6780659556388855, 0.5556982755661011, 0.20097464323043823, 0.5741451382637024, 0.7094172239303589, 0.5532907247543335, 0.49327176809310913, 0.7085888981819153, 0.57177734375, 0.5692128539085388, 0.6498693227767944, 0.6021918058395386, 0.5893454551696777, 0.7766348123550415, 0.6560738682746887, 0.6358551383018494, 0.6415812969207764, 0.734319269657135, 0.6573356986045837, 0.7262839078903198, 0.6914979815483093, 0.6619200110435486, 0.6852644085884094, 0.5410394072532654, 0.2743534743785858, 0.19936248660087585, 0.4735962152481079, 0.26864489912986755, 0.33471664786338806, 0.38149333000183105, 0.3993469476699829, 0.735224604

In [ ]:
print(questions_answers.equals(groups_df))

In [ ]:
best_answers_pred = (questions_answers.groupby(["QId"])["Cosine_Sim"].idxmax()).tolist()
best_answers_test = (questions_answers.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

In [ ]:
# Calculate accuracy
count=0
correct=0
for i in range(len(best_answers_test)):
  if best_answers_pred[i] == best_answers_test[i]:
    correct += 1
  count += 1
print(correct / count)

0.159001093079963


In [ ]:
q_sentence_embeddings.shape

(108705, 1024)

**USE THE SENTENCE EMBEDDINGS AND COSINE SIM TO TRAIN MODELS INSTEAD**

In [ ]:
import pandas as pd
questions_answers = pd.read_csv(os.path.join(project_folder, 'Train_w_cosine_sim.csv'))

In [ ]:
questions_answers['Q_Embedding'] = q_sentence_embeddings.tolist()
questions_answers['A_Embedding'] = a_sentence_embeddings.tolist()

NameError: ignored

In [ ]:
questions_answers

,Unnamed: 0,Unnamed: 0.1,Score,QId,Before,After,Cosine_to_Question,Cosine_to_Answers,Word_cnt,Char_cnt,Avg_char_length,Urls,Codes,Grade_Level,Dale_chall,Reading_Ease,Polarity,Subjectivity,Cumulative_ Answer_Score,Num_Answers,Num_Questions,Average_Answer_Score,Q_Word_Cnt,Q_Char_Cnt,Q_Avg_char_length,Q_Urls,Q_CodeSections,Q_GradeLevel,Q_Dale_chall,Q_ReadingEase,Q_Polarity,Q_Subjectivity,Clean_Question,Clean_Answer,Normalized_Score,Cosine_Sim,Q_Embedding,A_Embedding
0,0,0,1,180,0,8,0.140249,0.144123,279,1674,6.000000,0,2,10.205000,8.049667,57.520000,-0.132211,0.451375,144,21,1,6.857143,41,260,6.341463,0,0,3.168889,10.894532,87.911453,-0.132211,0.451375,this is something ive pseudosolved many times ...,ive read somewhere the human eye cant distingu...,0.047619,0.461888,"[0.8188315629959106, -0.6627430319786072, 0.65...","[0.468377947807312, -1.1847835779190063, 0.837..."
1,1,1,3,180,1,7,0.161577,0.117777,52,275,5.288462,0,0,2.216136,7.189409,103.664205,0.166667,0.500000,27,13,3,2.076923,41,260,6.341463,0,0,2.216136,7.189409,103.664205,0.166667,0.500000,this is something ive pseudosolved many times ...,isnt it also a factor which order you set up t...,0.142857,0.490419,"[0.8188315033912659, -0.6627426743507385, 0.65...","[0.07679979503154755, -0.8466829061508179, 0.2..."
2,2,2,21,180,2,6,0.142387,0.176084,190,1308,6.884211,5,0,7.489091,8.396844,71.324545,0.138636,0.416667,1738,23,1,75.565217,41,260,6.341463,0,0,7.489091,8.396844,71.324545,0.138636,0.416667,this is something ive pseudosolved many times ...,my first thought on this is how generate n vec...,1.000000,0.277971,"[0.8188315033912659, -0.6627426743507385, 0.65...","[0.11846833676099777, -0.6541638374328613, 0.6..."
3,3,3,17,180,3,5,0.060784,0.086928,55,370,6.727273,0,0,16.804091,12.464864,23.566136,0.100000,0.270000,28,14,0,2.000000,41,260,6.341463,0,0,16.804091,12.464864,23.566136,0.100000,0.270000,this is something ive pseudosolved many times ...,it would be best to find colors maximally dist...,0.809524,0.368167,"[0.8188315033912659, -0.6627426743507385, 0.65...","[0.2791283130645752, -0.3146314322948456, 0.15..."
4,4,4,7,180,4,4,0.192515,0.079836,46,467,10.152174,4,0,12.963913,11.605512,29.428188,0.233333,0.366667,449,53,3,8.471698,41,260,6.341463,0,0,12.963913,11.605512,29.428188,0.233333,0.366667,this is something ive pseudosolved many times ...,some related resources colorbrewer sets of co...,0.333333,0.293958,"[0.8188315629959106, -0.6627430319786072, 0.65...","[0.1279553771018982, -0.10148781538009644, 0.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108700,108700,108700,0,40133400,2,4,0.145302,0.088422,122,717,5.877049,0,2,7.155510,7.991327,72.852585,-0.141111,0.342778,342,72,7,4.750000,90,553,6.144444,0,0,5.912500,11.407792,70.038750,-0.141111,0.342778,i have an object now the problem is that i ca...,heres a sample showing how you can walk down a...,0.000000,0.550311,"[1.1249408721923828, -1.236080288887024, 0.286...","[0.7200525999069214, -0.7376412153244019, -0.3..."
108701,108701,108701,0,40133400,3,3,0.017753,0.081661,52,264,5.076923,0,1,-2.620000,0.148800,119.190000,-0.750000,1.000000,0,1,1,0.000000,90,553,6.144444,0,0,3.115369,11.464744,80.105862,-0.750000,1.000000,i have an object now the problem is that i ca...,this should work,0.000000,0.133255,"[1.1249409914016724, -1.2360790967941284, 0.28...","[0.8739054799079895, -0.5649404525756836, 0.77..."
108702,108702,108702,0,40133400,4,2,0.142839,0.066078,73,563,7.712329,0,1,10.978333,9.432317,59.100000,-0.280000,0.580000,336,241,0,1.394191,90,553,6.144444,0,0,2.828485,12.076895,82.555909,-0.280000,0.580000,i have an object now the problem is that i ca...,you could split the path and iterate it and re...,0.000000,0.363313,"[1.1249409914016724, -1.2360790967941284, 0.28...","[-0.7491970658302307, -0.29398468136787415, 0...."
108703,108703,108703,0,40133400,5,1,0.028369,0.086090,55,492,8.945455,0,1

In [ ]:
questions_answers.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Score', 'QId', 'Before', 'After',
       'Cosine_to_Question', 'Cosine_to_Answers', 'Word_cnt', 'Char_cnt',
       'Avg_char_length', 'Urls', 'Codes', 'Grade_Level', 'Dale_chall',
       'Reading_Ease', 'Polarity', 'Subjectivity', 'Cumulative_ Answer_Score',
       'Num_Answers', 'Num_Questions', 'Average_Answer_Score', 'Q_Word_Cnt',
       'Q_Char_Cnt', 'Q_Avg_char_length', 'Q_Urls', 'Q_CodeSections',
       'Q_GradeLevel', 'Q_Dale_chall', 'Q_ReadingEase', 'Q_Polarity',
       'Q_Subjectivity', 'Clean_Question', 'Clean_Answer', 'Normalized_Score',
       'Cosine_Sim', 'Q_Embedding', 'A_Embedding'],
      dtype='object')

In [ ]:
questions_answers = questions_answers.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

KeyboardInterrupt: ignored

In [ ]:
q_sentence_embeddings.shape

(108705, 1024)

**TRAINING A BUNCH OF MACHINE LEARNING MODELS USING THE EMBEDDINGS (Q AND A SEPARATE)**

In [ ]:
# CHOOSE WHETHER TO LOAD TENSOR OR NP ARRAY
# Load the sentence embeddings from disk
import pickle
import os

with open(os.path.join(project_folder, 'q_embeddings.pkl'), "rb") as fIn:
    stored_data = pickle.load(fIn)
    q_sentences = stored_data['sentences']
    q_sentence_embeddings = stored_data['embeddings']

# Load the sentence embeddings from disk
with open(os.path.join(project_folder, 'a_embeddings.pkl'), "rb") as fIn:
    stored_data = pickle.load(fIn)
    a_sentences = stored_data['sentences']
    a_sentence_embeddings = stored_data['embeddings']

pickle_qa_inp_path = os.path.join(project_folder, "bert_qa_embeddings.pkl")
qa_embeddings = pickle.load(open(pickle_qa_inp_path, 'rb'))

In [ ]:
import pandas as pd
questions_answers = pd.read_csv(os.path.join(project_folder, 'Train_RemovedNAN_AddedNormScore.csv'))

In [ ]:
# Find the split index, then iterate to next question since the the split should not be in the middle of a question
split_index = int(len(questions_answers) * 0.8)
split_Q = questions_answers['QId'].iloc[split_index]
while(split_index < len(questions_answers) and questions_answers['QId'].iloc[split_index] == split_Q):
  split_index += 1
questions_answers_train = questions_answers[:split_index]
questions_answers_test = questions_answers[split_index:]

In [ ]:
questions_answers

,Unnamed: 0,Score,QId,Before,After,Cosine_to_Question,Cosine_to_Answers,Word_cnt,Char_cnt,Avg_char_length,Urls,Codes,Grade_Level,Dale_chall,Reading_Ease,Polarity,Subjectivity,Cumulative_ Answer_Score,Num_Answers,Num_Questions,Average_Answer_Score,Q_Word_Cnt,Q_Char_Cnt,Q_Avg_char_length,Q_Urls,Q_CodeSections,Q_GradeLevel,Q_Dale_chall,Q_ReadingEase,Q_Polarity,Q_Subjectivity,Clean_Question,Clean_Answer,Normalized_Score
0,0,1,180,0,8,0.140249,0.144123,279,1674,6.000000,0,2,10.205000,8.049667,57.520000,-0.132211,0.451375,144,21,1,6.857143,41,260,6.341463,0,0,3.168889,10.894532,87.911453,-0.132211,0.451375,this is something ive pseudosolved many times ...,ive read somewhere the human eye cant distingu...,0.047619
1,1,3,180,1,7,0.161577,0.117777,52,275,5.288462,0,0,2.216136,7.189409,103.664204,0.166667,0.500000,27,13,3,2.076923,41,260,6.341463,0,0,2.216136,7.189409,103.664204,0.166667,0.500000,this is something ive pseudosolved many times ...,isnt it also a factor which order you set up t...,0.142857
2,2,21,180,2,6,0.142387,0.176084,190,1308,6.884211,5,0,7.489091,8.396844,71.324545,0.138636,0.416667,1738,23,1,75.565217,41,260,6.341463,0,0,7.489091,8.396844,71.324545,0.138636,0.416667,this is something ive pseudosolved many times ...,my first thought on this is how generate n vec...,1.000000
3,3,17,180,3,5,0.060784,0.086928,55,370,6.727273,0,0,16.804091,12.464864,23.566136,0.100000,0.270000,28,14,0,2.000000,41,260,6.341463,0,0,16.804091,12.464864,23.566136,0.100000,0.270000,this is something ive pseudosolved many times ...,it would be best to find colors maximally dist...,0.809524
4,4,7,180,4,4,0.192515,0.079836,46,467,10.152174,4,0,12.963913,11.605512,29.428188,0.233333,0.366667,449,53,3,8.471698,41,260,6.341463,0,0,12.963913,11.605512,29.428188,0.233333,0.366667,this is something ive pseudosolved many times ...,some related resources colorbrewer sets of co...,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108700,108700,0,40133400,2,4,0.145302,0.088422,122,717,5.877049,0,2,7.155510,7.991327,72.852585,-0.141111,0.342778,342,72,7,4.750000,90,553,6.144444,0,0,5.912500,11.407792,70.038750,-0.141111,0.342778,i have an object now the problem is that i ca...,heres a sample showing how you can walk down a...,0.000000
108701,108701,0,40133400,3,3,0.017753,0.081661,52,264,5.076923,0,1,-2.620000,0.148800,119.190000,-0.750000,1.000000,0,1,1,0.000000,90,553,6.144444,0,0,3.115369,11.464744,80.105862,-0.750000,1.000000,i have an object now the problem is that i ca...,this should work,0.000000
108702,108702,0,40133400,4,2,0.142839,0.066078,73,563,7.712329,0,1,10.978333,9.432317,59.100000,-0.280000,0.580000,336,241,0,1.394191,90,553,6.144444,0,0,2.828485,12.076895,82.555909,-0.280000,0.580000,i have an object now the problem is that i ca...,you could split the path and iterate it and re...,0.000000
108703,108703,0,40133400,5,1,0.028369,0.086090,55,492,8.945455,0,1,3.650000,5.711500,86.705000,-0.300000,0.712500,1,1,0,1.000000,90,553,6.144444,0,0,7.514857,13.287643,47.546000,-0.300000,0.712500,i have an object now the problem is that i ca...,try this it will work for whichever level you ...,0.000000


**MODELS WITH COSINE SIMILARITY PLUS PREVIOUS FEATURES**

In [ ]:
import numpy as np

Xtr = a_sentence_embeddings[0:split_index]
Ytr = questions_answers['Normalized_Score'][0:split_index]

Xte = a_sentence_embeddings[split_index:]
Yte = questions_answers['Normalized_Score'][split_index:]

X_new = questions_answers.drop(columns=['Unnamed: 0', 'Score', 'QId', 'Clean_Question', 'Clean_Answer', 'Normalized_Score'])
Xtr_new = X_new[0:split_index]
Xte_new = X_new[split_index:]

Xtr_cos = questions_answers_train['Cosine_Sim']
Xtr_cos = np.array(Xtr_cos)
Xtr_cos = Xtr_cos.reshape((Xtr_cos.shape[0], 1))

Xte_cos = questions_answers_test['Cosine_Sim']
Xte_cos = np.array(Xte_cos)
Xte_cos = Xte_cos.reshape((Xte_cos.shape[0], 1))

print(Xtr_cos.shape)
print(Xte_cos.shape)

Xtr = np.concatenate((Xtr_new, Xtr_cos), axis=1)
Xte = np.concatenate((Xte_new, Xte_cos), axis=1)

print(Xtr.shape, Xte.shape)

(86965, 1)
(21740, 1)
(86965, 30) (21740, 30)


In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(Xtr, Ytr)
predictions = reg.predict(Xte)

questions_answers_test['Predictions'] = predictions

best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Accuracy:  0.3245814307458143


In [ ]:
# MLP Regressor
from sklearn.neural_network import MLPRegressor

regr = MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=500, random_state=0).fit(Xtr, Ytr)
predictions = regr.predict(Xte)

questions_answers_test['Predictions'] = predictions

best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Accuracy:  0.3919330289193303


In [ ]:
# Calculate accuracy
def get_accuracy(prediction, test_set):
  count=0
  correct=0
  for i in range(len(test_set)):
    if prediction[i] == test_set[i]:
      correct += 1
    count += 1
  print("Accuracy: ", correct / count)

**MODELS WITH ANSWER EMBEDDINGS WITH OTHER FEATURES**

In [ ]:
Xtr = a_sentence_embeddings[0:split_index]
Ytr = questions_answers['Normalized_Score'][0:split_index]

Xte = a_sentence_embeddings[split_index:]
Yte = questions_answers['Normalized_Score'][split_index:]

In [ ]:
X_new = questions_answers.drop(columns=['Unnamed: 0', 'Score', 'QId', 'Clean_Question', 'Clean_Answer', 'Normalized_Score'])
Xtr_new = X_new[0:split_index]
Xte_new = X_new[split_index:]
Xtr = np.concatenate((Xtr, Xtr_new), axis=1)
Xte = np.concatenate((Xte, Xte_new), axis=1)
print(Xtr.shape, Xte.shape)

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(Xtr, Ytr)
predictions = reg.predict(Xte)
questions_answers_test['Predictions'] = predictions
best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Accuracy:  0.3177321156773212


In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=30, random_state=0)
regr.fit(Xtr, Ytr)
predictions=regr.predict(Xte)

best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

KeyboardInterrupt: ignored

**MODELS WITH QUESTIONS AND ANSWER EMBEDDINGS AND OTHER FEATURES**

In [ ]:
Xtr = np.concatenate((q_sentence_embeddings[0:split_index], a_sentence_embeddings[0:split_index]), axis=1)
Ytr = questions_answers['Normalized_Score'][0:split_index]

Xte = np.concatenate((q_sentence_embeddings[split_index:], a_sentence_embeddings[split_index:]), axis=1)
Yte = questions_answers['Normalized_Score'][split_index:]

Xtr.shape

(86965, 2048)

In [ ]:
Xtr_df = pd.DataFrame(data=Xtr)

In [ ]:
Xte_df = pd.DataFrame(data=Xte)

In [ ]:
Xte_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.392925,-0.243445,0.219118,0.398218,-0.136999,-0.434771,-0.196763,-0.028433,0.318256,0.694101,-0.573409,-0.524391,-0.223008,-0.863992,-0.727280,-1.205944,0.493870,0.124024,-0.234416,0.324723,0.274491,-0.069957,-0.912625,-0.048503,-1.098631,-0.720606,0.181879,-1.324041,0.133129,0.817167,0.120153,0.278016,-0.784025,-0.931402,-0.439767,-0.510777,-0.228426,0.216901,-0.144259,-0.181235,...,0.589800,-0.287599,0.436638,-0.485899,-1.278794,-0.017135,-0.228935,1.307602,0.827285,0.459137,-0.741439,-0.268476,-0.230023,-0.672036,-0.436774,1.620367,0.271842,-0.125618,-0.070511,0.508170,0.995512,-0.645879,0.183840,0.241788,-0.111340,-0.163375,-0.403056,1.230693,-0.053416,-0.288844,0.791636,0.205239,0.762193,0.497974,0.326350,0.074195,0.724724,-0.340791,0.396685,0.510020
1,0.392925,-0.243445,0.219118,0.398218,-0.136999,-0.434771,-0.196763,-0.028433,0.318256,0.694101,-0.573409,-0.524391,-0.223008,-0.863992,-0.727280,-1.205944,0.493870,0.124024,-0.234416,0.324723,0.274491,-0.069957,-0.912625,-0.048503,-1.098631,-0.720606,0.181879,-1.324041,0.133129,0.817167,0.120153,0.278016,-0.784025,-0.931402,-0.439767,-0.510777,-0.228426,0.216901,-0.144259,-0.181235,...,0.320851,0.173402,-0.207455,-0.028583,0.127483,-0.874203,0.336497,0.354835,0.395321,-0.356586,0.331123,-1.298380,0.831046,-0.093312,-1.118620,1.195031,-0.237901,0.124641,-0.405544,-0.006519,1.013075,0.146080,0.170758,0.097694,0.486258,-0.465178,0.064506,0.506913,-0.811255,0.033002,0.682108,0.469503,0.457392,-0.114854,0.754580,0.384680,0.975278,-0.088448,0.613372,-0.453958
2,0.392925,-0.243445,0.219118,0.398218,-0.136999,-0.434771,-0.196763,-0.028433,0.318256,0.694101,-0.573409,-0.524391,-0.223008,-0.863992,-0.727280,-1.205944,0.493870,0.124024,-0.234416,0.324723,0.274491,-0.069957,-0.912625,-0.048503,-1.098631,-0.720606,0.181879,-1.324041,0.133129,0.817167,0.120153,0.278016,-0.784025,-0.931402,-0.439767,-0.510777,-0.228426,0.216901,-0.144259,-0.181235,...,0.525474,-0.347323,0.518779,0.189733,-0.454688,-0.358255,0.033339,0.405669,0.336388,0.247556,-0.732348,-0.117048,0.339265,-0.720778,-0.445015,0.884974,-0.359645,-0.067904,0.043632,0.113936,0.805444,-0.320709,0.141513,-0.372027,-0.183830,0.731578,-0.055227,0.937442,0.125510,-0.585091,1.103516,0.479547,1.029995,0.180839,0.619322,0.196720,1.243153,-0.070065,-0.008552,-0.297769
3,0.392925,-0.243445,0.219118,0.398218,-0.136999,-0.434771,-0.196763,-0.028433,0.318256,0.694101,-0.573409,-0.524391,-0.223008,-0.863992,-0.727280,-1.205944,0.493870,0.124024,-0.234416,0.324723,0.274491,-0.069957,-0.912625,-0.048503,-1.098631,-0.720606,0.181879,-1.324041,0.133129,0.817167,0.120153,0.278016,-0.784025,-0.931402,-0.439767,-0.510777,-0.228426,0.216901,-0.144259,-0.181235,...,0.699166,-0.831628,-0.438367,-0.036360,0.372637,-0.396584,0.352560,-0.668757,1.257625,0.895753,-1.079231,-0.933211,0.845418,-0.234238,-0.204895,0.395051,-0.405028,0.617410,1.132755,-0.241962,1.336677,-0.266173,0.547925,-0.472723,0.341892,-0.289306,0.112659,0.625401,-0.138532,0.101391,-0.249188,-0.608998,1.126551,0.279400,0.428925,-0.632312,0.916329,0.047070,1.065005,0.387921
4,0.392925,-0.243445,0.219118,0.398218,-0.136999,-0.434771,-0.196763,-0.028433,0.318256,0.694101,-0.573409,-0.524391,-0.223008,-0.863992,-0.727280,-1.205944,0.493870,0.124024,-0.234416,0.324723,0.274491,-0.069957,-0.912625,-0.048503,-1.098631,-0.720606,0.181879,-1.324041,0.133129,0.817167,0.120153,0.278016,-0.784025,-0.931402,-0.439767,-0.510777,-0.228426,0.216901,-0.144259,-0.181235,...,0.738631,0.313884,1.357238,-0.258591,-0.537402,-0.545644,0.363532,0.623521,-0.426304,-0.279994,-0.780986,-0.177132,0.023126,-0.407621,0.008094,1.390016,-0.317086,0.340035,-0.120054,-0.065778,0.788388,0.003350,0.218828,0.1

In [ ]:
X_new = questions_answers.drop(columns=['Unnamed: 0', 'Score', 'QId', 'Clean_Question', 'Clean_Answer', 'Normalized_Score'])
Xtr_new = X_new[0:split_index]
Xte_new = X_new[split_index:]
Xtr = np.concatenate((Xtr_df, Xtr_new), axis=1)
Xte = np.concatenate((Xte_df, Xte_new), axis=1)
#Xtr = Xtr_df
#Xte = Xte_df
print(Xtr.shape, Xte.shape)

(86965, 2077) (21740, 2077)


In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(Xtr, Ytr)
predictions = reg.predict(Xte)

questions_answers_test['Predictions'] = predictions

best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Accuracy:  0.317351598173516


In [ ]:
# MLP Regressor
from sklearn.neural_network import MLPRegressor

regr = MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=500, random_state=0).fit(Xtr, Ytr)
predictions = regr.predict(Xte)

questions_answers_test['Predictions'] = predictions

best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Accuracy:  0.3732876712328767


In [ ]:
questions_answers

,Unnamed: 0,Score,QId,Before,After,Cosine_to_Question,Cosine_to_Answers,Word_cnt,Char_cnt,Avg_char_length,Urls,Codes,Grade_Level,Dale_chall,Reading_Ease,Polarity,Subjectivity,Cumulative_ Answer_Score,Num_Answers,Num_Questions,Average_Answer_Score,Q_Word_Cnt,Q_Char_Cnt,Q_Avg_char_length,Q_Urls,Q_CodeSections,Q_GradeLevel,Q_Dale_chall,Q_ReadingEase,Q_Polarity,Q_Subjectivity,Clean_Question,Clean_Answer,Normalized_Score
0,0,1,180,0,8,0.140249,0.144123,279,1674,6.000000,0,2,10.205000,8.049667,57.520000,-0.132211,0.451375,144,21,1,6.857143,41,260,6.341463,0,0,3.168889,10.894532,87.911453,-0.132211,0.451375,this is something ive pseudosolved many times ...,ive read somewhere the human eye cant distingu...,0.047619
1,1,3,180,1,7,0.161577,0.117777,52,275,5.288462,0,0,2.216136,7.189409,103.664204,0.166667,0.500000,27,13,3,2.076923,41,260,6.341463,0,0,2.216136,7.189409,103.664204,0.166667,0.500000,this is something ive pseudosolved many times ...,isnt it also a factor which order you set up t...,0.142857
2,2,21,180,2,6,0.142387,0.176084,190,1308,6.884211,5,0,7.489091,8.396844,71.324545,0.138636,0.416667,1738,23,1,75.565217,41,260,6.341463,0,0,7.489091,8.396844,71.324545,0.138636,0.416667,this is something ive pseudosolved many times ...,my first thought on this is how generate n vec...,1.000000
3,3,17,180,3,5,0.060784,0.086928,55,370,6.727273,0,0,16.804091,12.464864,23.566136,0.100000,0.270000,28,14,0,2.000000,41,260,6.341463,0,0,16.804091,12.464864,23.566136,0.100000,0.270000,this is something ive pseudosolved many times ...,it would be best to find colors maximally dist...,0.809524
4,4,7,180,4,4,0.192515,0.079836,46,467,10.152174,4,0,12.963913,11.605512,29.428188,0.233333,0.366667,449,53,3,8.471698,41,260,6.341463,0,0,12.963913,11.605512,29.428188,0.233333,0.366667,this is something ive pseudosolved many times ...,some related resources colorbrewer sets of co...,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108700,108700,0,40133400,2,4,0.145302,0.088422,122,717,5.877049,0,2,7.155510,7.991327,72.852585,-0.141111,0.342778,342,72,7,4.750000,90,553,6.144444,0,0,5.912500,11.407792,70.038750,-0.141111,0.342778,i have an object now the problem is that i ca...,heres a sample showing how you can walk down a...,0.000000
108701,108701,0,40133400,3,3,0.017753,0.081661,52,264,5.076923,0,1,-2.620000,0.148800,119.190000,-0.750000,1.000000,0,1,1,0.000000,90,553,6.144444,0,0,3.115369,11.464744,80.105862,-0.750000,1.000000,i have an object now the problem is that i ca...,this should work,0.000000
108702,108702,0,40133400,4,2,0.142839,0.066078,73,563,7.712329,0,1,10.978333,9.432317,59.100000,-0.280000,0.580000,336,241,0,1.394191,90,553,6.144444,0,0,2.828485,12.076895,82.555909,-0.280000,0.580000,i have an object now the problem is that i ca...,you could split the path and iterate it and re...,0.000000
108703,108703,0,40133400,5,1,0.028369,0.086090,55,492,8.945455,0,1,3.650000,5.711500,86.705000,-0.300000,0.712500,1,1,0,1.000000,90,553,6.144444,0,0,7.514857,13.287643,47.546000,-0.300000,0.712500,i have an object now the problem is that i ca...,try this it will work for whichever level you ...,0.000000


**GENERATE COMBINED Q-A EMBEDDINGS INSTEAD OF SEPARATE ONES AND TRAIN MODELS**

In [ ]:
Xtr = qa_embeddings[0:split_index]
Ytr = questions_answers['Normalized_Score'][0:split_index]

Xte = qa_embeddings[split_index:]
Yte = questions_answers['Normalized_Score'][split_index:]

In [ ]:
import numpy as np

X_new = questions_answers.drop(columns=['Unnamed: 0', 'Score', 'QId', 'Clean_Question', 'Clean_Answer', 'Normalized_Score'])

Xtr_new = X_new[0:split_index]
Xte_new = X_new[split_index:]

Xtr = np.concatenate((Xtr, Xtr_new), axis=1)
Xte = np.concatenate((Xte, Xte_new), axis=1)
print(Xtr.shape, Xte.shape)

(86965, 1052) (21740, 1052)


In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(Xtr, Ytr)
predictions = reg.predict(Xte)

questions_answers_test['Predictions'] = predictions

best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

print(Xtr.shape, Xte.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Accuracy:  0.3158295281582953
(86965, 1052) (21740, 1052)


In [ ]:
# MLP Regressor
from sklearn.neural_network import MLPRegressor

regr = MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=500, random_state=0).fit(Xtr, Ytr)
predictions = regr.predict(Xte)

questions_answers_test['Predictions'] = predictions

best_answers_pred = (questions_answers_test.groupby(["QId"])["Predictions"].idxmax()).tolist()
best_answers_test = (questions_answers_test.groupby(["QId"])["Normalized_Score"].idxmax()).tolist()

get_accuracy(best_answers_pred, best_answers_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Accuracy:  0.3896499238964992
